In [1]:
import os
import numpy as np
import PIL.Image
import random
import string
import shutil
from tqdm import tqdm
import cv2
import torch
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xywhn2xyxy, xyxy2xywhn
import pandas as pd

### Get anchor of the model

In [2]:
def get_anchor():
    from models.experimental import attempt_load  # scoped to avoid circular import
    w = '/Users/zhenyu/Desktop/results/bundle-ebc9656257714d73855f9d5264f47d51/weights/best.pt'
    model_pt = torch.jit.load(w) if 'torchscript' in w else attempt_load(w)
    print('Anchors are:\n {}'.format(model_pt.model[-1].anchors))
    print(model_pt.model[-1].grid[0].shape)
    print(model_pt.model[-1].grid[1].shape)
    print(model_pt.model[-1].grid[2].shape)
    print(model_pt.model[-1].grid[3].shape)
get_anchor()

Fusing layers... 
Model Summary: 444 layers, 86173414 parameters, 0 gradients, 204.0 GFLOPs


Anchors are:
 tensor([[[ 1.25000,  1.62500],
         [ 2.00000,  3.75000],
         [ 4.12500,  2.87500]],

        [[ 1.87500,  3.81250],
         [ 3.87500,  2.81250],
         [ 3.68750,  7.43750]],

        [[ 3.62500,  2.81250],
         [ 4.87500,  6.18750],
         [11.65625, 10.18750]]])
torch.Size([1, 3, 244, 184, 2])
torch.Size([1, 3, 122, 92, 2])
torch.Size([1, 3, 61, 46, 2])


IndexError: list index out of range

### Crop image to 256*256 with screw at the center

In [18]:
# crop images to 256*256 with screw at the center
IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG'
OUT_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Cropped/11111/'

i=0
for folder in [each for each in os.listdir(IMG_FOLDER) if not each.startswith('.') and each.startswith('3')]:
    for img in [img for img in os.listdir(os.path.join(IMG_FOLDER, folder)) if img.endswith('.jpg')]:
        with open(os.path.join(IMG_FOLDER, folder, '{}.txt'.format(img[:-4])), 'r') as f:
            lines = f.read().splitlines()
        image = PIL.Image.open(os.path.join(IMG_FOLDER, folder, img))
        for line in lines:
            x, y = [float(num) for num in line.split(' ')[1:3]]
            x = int(x*3024)
            y = int(y*4032)
            cropped = image.crop((x-128, y-128, x+128, y+128))
            name = OUT_FOLDER + ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)) + '.jpg'
            cropped.save(name)
            i+=1
            if i >= 150:
                raise NameError('TOOOOO MANYYYYY')

NameError: TOOOOO MANYYYYY

### Remove cropped images without json and rename the jpg&json

In [195]:
# get rid of cropped images without json and rename the jpg&json

IMG_FOLDER = '/Users/zhenyu/Desktop/1111_Zhenyu/'
JSON_FOLDER = IMG_FOLDER
i=0
for img in [file for file in os.listdir(IMG_FOLDER) if file.endswith('.jpg')]:
    json = '{}.json'.format(img[:-4])
    if os.path.exists(os.path.join(JSON_FOLDER, json)):
        os.rename(os.path.join(IMG_FOLDER, img), os.path.join(IMG_FOLDER, '{}.jpg'.format(i)))
        os.rename(os.path.join(JSON_FOLDER, json), os.path.join(JSON_FOLDER, '{}.json'.format(i)))
        i += 1
    else:
        os.remove(os.path.join(IMG_FOLDER, img))
        print('File: {} removed!'.format(img))

File: 11111_20_11.jpg removed!
File: 11111_1_16.jpg removed!
File: 11111_20_16.jpg removed!
File: 11111_1_39.jpg removed!
File: 11111_0_1.jpg removed!
File: 11111_0_22.jpg removed!
File: 11111_21_30.jpg removed!
File: 11111_21_31.jpg removed!
File: 11111_0_7.jpg removed!
File: 11111_0_18.jpg removed!
File: 11111_21_34.jpg removed!
File: 11111_21_36.jpg removed!
File: 11111_21_37.jpg removed!
File: 11111_20_46.jpg removed!
File: 11111_1_47.jpg removed!
File: 11111_1_35.jpg removed!
File: 11111_20_19.jpg removed!
File: 11111_20_32.jpg removed!
File: 11111_20_5.jpg removed!
File: 11111_20_33.jpg removed!


### Select OK images randomly to destination

In [25]:
# randomly select OK images to destination and rename

IMG_FOLDER = '/Users/zhenyu/Desktop/AOI7/raw_filtered'
OUT_FOLDER = '/Users/zhenyu/Desktop/AOI7/OK'

img_list = []
for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            img_list.append(os.path.join(root, name))
rand_img_list = random.sample(img_list, 1500)
for file in tqdm(rand_img_list):
    shutil.move(file, os.path.join(OUT_FOLDER, file.split('/')[-2], file.split('/')[-1]))

100%|██████████| 1499/1499 [00:00<00:00, 1892.57it/s]


### Move labelled data (from LabelImg) to test folder

In [3]:
# move labelled data to test folder

IMG_FOLDER = '/Users/zhenyu/Downloads/NG'
TEST_FOLDER = '/Users/zhenyu/Desktop/validation/'

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        label = os.path.join(root, '{}.txt'.format(name[:-4]))
        if name.endswith('.jpg') and os.path.exists(label):
            shutil.copyfile(os.path.join(root, name), os.path.join(TEST_FOLDER, 'images/test', name))
            shutil.copyfile(label, os.path.join(TEST_FOLDER, 'labels/test', '{}.txt'.format(name[:-4])))

### Move synthesized data to test folder

In [ ]:
# move labelled data to test folder

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/Artificial_Screw_Generation/ImageGeneration20211217/Station1/Batch1/Images/'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/Artificial_Screw_Generation/ImageGeneration20211217/Station1/Batch1/Yolo_Format/'

IMG_DST = '/Users/zhenyu/Desktop/dataset_1228/images/train/'
LABEL_DST = '/Users/zhenyu/Desktop/dataset_1228/labels/train/'

IMG_LIST = []
LABEL_LIST = []

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            IMG_LIST.append(name[:-4])
        if name.endswith('.txt'):
            LABEL_LIST.append(name[:-4])
for name in tqdm(IMG_LIST):
    if name in LABEL_LIST:
        shutil.copyfile(os.path.join(IMG_FOLDER, '{}.jpg'.format(name)), os.path.join(IMG_DST, '{}.jpg'.format(name)))
        shutil.copyfile(os.path.join(LABEL_FOLDER, '{}.txt'.format(name)), os.path.join(LABEL_DST, '{}.txt'.format(name)))

### Remove images without labels

In [29]:
# get rid of images without labels

IMG_FOLDER = '/Users/zhenyu/Desktop/AOI9/Real/images/'
LABEL_FOLDER = '/Users/zhenyu/Desktop/AOI9/Real/labels/'
i=0
for img in [file for file in os.listdir(IMG_FOLDER) if file.endswith('.jpg')]:
    label = '{}.txt'.format(img[:-4])
    if os.path.exists(os.path.join(LABEL_FOLDER, label)):
#         os.rename(os.path.join(IMG_FOLDER, img), os.path.join(IMG_FOLDER, '{}.jpg'.format(i)))
#         os.rename(os.path.join(LABEL_FOLDER, label), os.path.join(LABEL_FOLDER, '{}.txt'.format(i)))
#         i += 1
        continue
    else:
        os.remove(os.path.join(IMG_FOLDER, img))
        print('File: {} removed!'.format(img))
for label in [file for file in os.listdir(LABEL_FOLDER) if file.endswith('.txt')]:
    img = '{}.jpg'.format(label[:-4])
    if os.path.exists(os.path.join(IMG_FOLDER, img)) or (img == 'classes.jpg'):
        continue
    else:
        os.remove(os.path.join(LABEL_FOLDER, label))
        print('File: {} removed!'.format(label))

File: J314.4.AOI9.20220228.162557.xml.txt removed!
File: J314.3.AOI9.FOV4.20220225.185533.xml.txt removed!
File: J314.4.AOI9.FOV1.20220225.183457.xml.txt removed!
File: J314.3.AOI9.FOV4.20220226.182505.xml.txt removed!
File: J314.4.AOI9.FOV1.20220225.185541.xml.txt removed!
File: J314.38.AOI9.20220228.173555.xml.txt removed!
File: J314.3.AOI9.FOV4.20220226.181312.xml.txt removed!
File: J314.3.AOI9.FOV4.20220225.191419.xml.txt removed!
File: J314.4.AOI9.FOV1.20220226.180739.xml.txt removed!
File: J314.3.AOI9.FOV4.20220226.184602.xml.txt removed!
File: J314.10.AOI9.FOV1.20220225.193124.xml.txt removed!
File: J314.3.AOI9.FOV4.20220226.182844.xml.txt removed!
File: J314.4.AOI9.FOV1.20220225.193017.xml.txt removed!
File: J314.3.AOI9.FOV4.20220225.193007.xml.txt removed!


### Draw Bounding Box

In [ ]:
# draw bounding box

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG/33333_20211209111350/'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG/33333_20211209111350/'

def draw_bb(image, label, color=(0,0,255)):
    img = cv2.imread(IMG_FOLDER+image)
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        rec_img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=2)
    cv2.imwrite(IMG_FOLDER+image[:-4]+'_test.jpg', rec_img)

for image in [f for f in os.listdir(IMG_FOLDER) if f.endswith('.jpg')]:
    try:
        with open(LABEL_FOLDER+image[:-4]+'.txt') as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
        draw_bb(image, label)
    except FileNotFoundError:
        print('File Not Found')
        pass

### Hightlight overkills&escapes

In [14]:
# highlight all overkilled and escaped objects

IMG_FOLDER = '/Users/zhenyu/Desktop/AOI7/Real/images/'
LABEL_FOLDER = '/Users/zhenyu/Desktop/AOI7/Real/labels/'
PRED_FOLDER = '/Users/zhenyu/Desktop/AOI7/Real/images/Output/'
OUT_FOLDER = '/Users/zhenyu/Desktop/overkill/'

def iou_calc(box1, box2, eps=1e-7):
    
    box2 = box2.T
    
    b1_x1, b1_x2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
    b1_y1, b1_y2 = box1[2] - box1[4] / 2, box1[2] + box1[4] / 2
    b2_x1, b2_x2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2
    b2_y1, b2_y2 = box2[2] - box2[4] / 2, box2[2] + box2[4] / 2
    
    inter = (np.minimum(b1_x2, b2_x2) - np.maximum(b1_x1, b2_x1)).clip(0) * \
            (np.minimum(b1_y2, b2_y2) - np.maximum(b1_y1, b2_y1)).clip(0)

    # Union Area
    w1, h1 = box1[3], box1[4] + eps
    w2, h2 = box2[3], box2[4] + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union
    return iou

def draw_bb_iou(img, label, color=(0,0,255), text=''):
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        img = cv2.rectangle(img, (x_min-6, y_min-6), (x_max+6, y_max+6), color=color, thickness=4)
        cv2.putText(img, text, (x_min-10, y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 2, color, 6)
#         cv2.putText(img, text, (500, 500), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 4)

    return img

n_label = 0
n_pred = 0
n_FP = 0
n_FN = 0
TP = []
FN = []
FOV={'FOV1':0, 'FOV2':0, 'FOV3':0, 'FOV4':0}

for image in tqdm([img for img in os.listdir(IMG_FOLDER) if img.endswith('.jpg')]):
    try:
        with open(os.path.join(LABEL_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        label = []
    try:
        with open(os.path.join(PRED_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            pred = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        pred = []
    label.sort()
    pred.sort()
    label = np.array(label)
    n_label += label.shape[0]
    pred = np.array(pred)
    n_pred += pred.shape[0]
    index_label_list = []
    
    img = cv2.imread(os.path.join(IMG_FOLDER, image))
    for index_label, value in enumerate(label):
        iou = iou_calc(value, pred)
        try:
            iou_max, index_pred = torch.max(torch.tensor(iou), 0)
        except IndexError:
            continue
        if iou_max >= 1e-5:
            if pred[index_pred][-1] >=0.5:
                img = draw_bb_iou(img, [value], color=(0,0,128))
                TP.append(pred[index_pred][5])
            elif pred[index_pred][-1] >=0.3:
                img = draw_bb_iou(img, [value], color=(255,144,30))
                TP.append(pred[index_pred][5])
            pred = np.delete(pred, index_pred, 0)
            index_label_list.append(index_label)
# uncomment if need to count label numbers in each FOV
#     for fov in ['FOV1', 'FOV2', 'FOV3', 'FOV4']:
#         if fov in image:
#             FOV[fov] += len(label)

    label = np.delete(label, index_label_list, 0)
    
    n_FN += label.shape[0]
    n_FP += pred.shape[0]
    
    img = draw_bb_iou(img, pred[:,:-1], color=(0,215,255), text='OVERKILL')
    img = draw_bb_iou(img, label, color=(139,0,139), text='ESCAPE')
# uncomment if need to count escaped in each FOV
#     if np.any(label):
#         for fov in ['FOV1', 'FOV2', 'FOV3', 'FOV4']:
#             if fov in image:
#                 FOV[fov] += len(label)
#                 break
    
#     if np.any(label) or np.any(pred):
#         cv2.imwrite(os.path.join(OUT_FOLDER, '{}_iou.jpg'.format(image[:-4])), img)
print('Precision is {:%}'.format((n_pred-n_FP)/n_pred))
print('Recall is {:%}'.format((n_pred-n_FP)/n_label))
print('Escape rate is {:%}'.format(1-(n_pred-n_FP)/n_label))
print('Overkill rate is {:%}'.format((n_FP)/n_label))

100%|██████████| 315/315 [00:42<00:00,  7.37it/s]

Precision is 98.474086%
Recall is 99.345307%
Escape rate is 0.654693%
Overkill rate is 1.539414%


In [184]:
# highlight all overkilled and escaped objects

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Overkill/Raw/FOV3_136'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Overkill/Raw/FOV3_136'
PRED_FOLDER = '/Users/zhenyu/Desktop/FOV3_136'
OUT_FOLDER = '/Users/zhenyu/Desktop/iou'

def iou_calc(box1, box2, eps=1e-7):
    
    box2 = box2.T
    
    b1_x1, b1_x2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
    b1_y1, b1_y2 = box1[2] - box1[4] / 2, box1[2] + box1[4] / 2
    b2_x1, b2_x2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2
    b2_y1, b2_y2 = box2[2] - box2[4] / 2, box2[2] + box2[4] / 2
    
    inter = (np.minimum(b1_x2, b2_x2) - np.maximum(b1_x1, b2_x1)).clip(0) * \
            (np.minimum(b1_y2, b2_y2) - np.maximum(b1_y1, b2_y1)).clip(0)

    # Union Area
    w1, h1 = box1[3], box1[4] + eps
    w2, h2 = box2[3], box2[4] + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union
    return iou

def draw_bb_iou(img, label, color=(0,0,255), text=''):
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=4)
        cv2.putText(img, text, (x_min, y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 4)
    
    return img

n_label = 0
n_pred = 0
n_FP = 0
n_FN = 0
# TP = []
# FN = []
MESA = []

for image in tqdm([img for img in os.listdir(IMG_FOLDER) if img.endswith('.jpg')]):
    try:
        with open(os.path.join(LABEL_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        label = []
    try:
        with open(os.path.join(PRED_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            pred = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        pred = []

    label.sort()
    pred.sort()
    label = np.array(label)
    n_label += label.shape[0]
    pred = np.array(pred)
    n_pred += pred.shape[0]
    index_label_list = []
    
    for index_label, value in enumerate(pred):
        if 0.1256<value[1]<0.33 and 0.62<value[2]<0.7565:
            MESA.append(value[5])
    
#     img = cv2.imread(os.path.join(IMG_FOLDER, image))
#     for index_label, value in enumerate(label):
#         iou = iou_calc(value, pred[:-1])
#         try:
#             iou_max, index_pred = torch.max(torch.tensor(iou), 0)
#         except IndexError:
#             continue
#         if iou_max >= 0.5:
# #             img = draw_bb_iou(img, [value], color=(0,255,0))
#             TP.append(pred[index_pred][5])
#             pred = np.delete(pred, index_pred, 0)
#             index_label_list.append(index_label)
#         elif iou_max >= 0.3:
# #             img = draw_bb_iou(img, [value], color=(255,0,0), text='Low IOU')
#             TP.append(pred[index_pred][5])
#             pred = np.delete(pred, index_pred, 0)
#             index_label_list.append(index_label)
#     label = np.delete(label, index_label_list, 0)
    
#     n_FN += label.shape[0]
#     n_FP += pred.shape[0]
#     for each in pred:
#         FN.append(each[5])
    
# #     img = draw_bb_iou(img, label, color=(0,0,255), text='Escape')
# #     img = draw_bb_iou(img, pred, color=(0,0,0), text='Overkill')
            
# #     cv2.imwrite(os.path.join(OUT_FOLDER, '{}_iou.jpg'.format(image[:-4])), img)
# print('Precision is {:%}'.format((n_pred-n_FP)/n_pred))
# print('Recall is {:%}'.format((n_pred-n_FP)/n_label))
# print('Escape rate is {:%}'.format(1-(n_pred-n_FP)/n_label))
# print('Overkill rate is {:%}'.format((n_FP)/n_label))

100%|██████████| 136/136 [00:00<00:00, 2702.27it/s]


In [188]:
import seaborn as sns
df = pd.DataFrame([FN, TP, MESA]).transpose()
df = df.rename(columns={0:'FN', 1:'TP', 2:'MESA'})
df['id'] = df.index
df = pd.melt(df, value_vars=['FN', 'TP', 'MESA'])
sns.displot(df, x='value', hue='variable', kind='hist', height=10)
plt.savefig('/Users/zhenyu/Desktop/hist.png')

### Remove Images with empty label

In [208]:
IMG_FOLDER = '/Users/zhenyu/Desktop/OK_label'

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            txt_name = '{}.txt'.format(name[:-4])
            try:
                with open(os.path.join(root,txt_name)) as f:
                    label = [list(map(float, line.rstrip().split())) for line in f]
                if not label:
                    os.remove(os.path.join(root,txt_name))
                    os.remove(os.path.join(root,name))
            except FileNotFoundError:
                os.remove(os.path.join(root,name))

### Replace labels with 0

In [63]:
LABEL_FOLDER = '/Users/zhenyu/Desktop/AOI9/Real/images/'

for file in [file for file in os.listdir(LABEL_FOLDER) if file.endswith('txt') and ('class' not in file)]:
    with open(os.path.join(LABEL_FOLDER, file)) as f:
        label = [list(map(float, line.rstrip().split())) for line in f]
    for each in label:
        each[0] = 0
    with open(os.path.join(LABEL_FOLDER, file), 'w') as fout:
        fout.write('\n'.join([' '.join(map(str, line)) for line in label]))

In [199]:
def condition(x):
    return x > 0.7
1- sum(condition(x) for x in MESA)/len(MESA)

0.948051948051948

### Unzip Insight data and move to destination

In [7]:
import tarfile
FOLDER_PATH = ['/Users/zhenyu/Downloads/data_9/',
              '/Users/zhenyu/Downloads/data_10/',
              '/Users/zhenyu/Downloads/data_11/',
              '/Users/zhenyu/Downloads/data_12/',
              '/Users/zhenyu/Downloads/data_13/',
              '/Users/zhenyu/Downloads/data_14/',
              '/Users/zhenyu/Downloads/data_15/',]
DST_PATH = '/Users/zhenyu/Desktop/AOI7/OK_Val/'
for subdir in ['FOV1', 'FOV2', 'FOV3', 'FOV4']:
    directory = os.path.join(DST_PATH, subdir)
    if not os.path.exists(directory):
        os.makedirs(directory)

for folder in tqdm(FOLDER_PATH):
    for root, dirs, files in os.walk(folder):
        for zipfile in files:
            if zipfile.endswith('.tgz'):
                tar = tarfile.open(os.path.join(root, zipfile), "r:gz")
                tar.extractall(path=root)
                tar.close()
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('jpg') and ('result' not in file) and ('FOV1' in file):
                shutil.move(os.path.join(root,file), os.path.join(DST_PATH, 'FOV1', file))
            elif file.endswith('jpg') and ('result' not in file) and ('FOV2' in file):
                shutil.move(os.path.join(root,file), os.path.join(DST_PATH, 'FOV2', file))
            elif file.endswith('jpg') and ('result' not in file) and ('FOV3' in file):
                shutil.move(os.path.join(root,file), os.path.join(DST_PATH, 'FOV3', file))
            elif file.endswith('jpg') and ('result' not in file) and ('FOV4' in file):
                shutil.move(os.path.join(root,file), os.path.join(DST_PATH, 'FOV4', file))

100%|██████████| 7/7 [08:58<00:00, 76.90s/it]


### Filter and Move Images Based on Pattern Match Results

In [50]:
import math
CSV_PATH = '/Users/zhenyu/Desktop/AOI7/raw/FOV1/Output/PatternMatch_Output.csv'
FOV = ['FOV1', 'FOV2', 'FOV3', 'FOV4']
IMG_PATH = '/Users/zhenyu/Desktop/AOI7/raw/'
DST_PATH = '/Users/zhenyu/Desktop/AOI7/OK_Val/'
result = pd.read_csv(CSV_PATH)
result['angle'] = result.apply(lambda row: math.atan2(row.y1-row.y2, row.x1-row.x2)*180.0/math.pi , axis=1)
result_filtered = result.loc[lambda result: (abs(result.x1 - 2530) <= 40) & 
                             (abs(result.y1 - 3280) <= 40) &
                             (abs(result.x2 - 1130) <= 40) & 
                             (abs(result.y2 - 2100) <= 40) & 
                             (abs(result.angle - 40.1) <= 1)]
for subdir in FOV:
    directory = os.path.join(DST_PATH, subdir)
    if not os.path.exists(directory):
        os.makedirs(directory)
        

for root, dirs, files in os.walk(IMG_PATH):
    for file in files:
        if result_filtered['No'].str.contains(file[:27]).any():
            shutil.move(os.path.join(root,file), 
                        os.path.join(DST_PATH, root.split('/')[-1], file))

### Move FIT images with labels and add .FOV in name

In [31]:
path = '/Users/zhenyu/Downloads/Station2'
dst = '/Users/zhenyu/Desktop/AOI7/Real'
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.jpg'):
            shutil.move(os.path.join(root,file), os.path.join(dst, 'images',file[:-4]+'.'+root.split('/')[-1]+file[-4:]))
        elif file.endswith('.txt') and 'classes' not in file:
            shutil.move(os.path.join(root,file), os.path.join(dst, 'labels',file[:-4]+'.'+root.split('/')[-1]+file[-4:]))

In [71]:
from datetime import datetime, date
with open('/Users/zhenyu/Downloads/Loop2Info_2.txt') as file:
    start_insert = [datetime.strptime(line.rstrip()[7:19], '%H:%M:%S.%f').time() for line in file if '2522' in line]
with open('/Users/zhenyu/Downloads/Loop2Info_2.txt') as file:
    end_insert = [datetime.strptime(line.rstrip()[7:19], '%H:%M:%S.%f').time()  for line in file if '2525' in line]
with open('/Users/zhenyu/Downloads/Loop2Info_2.txt') as file:
    start = [datetime.strptime(line.rstrip()[7:19], '%H:%M:%S.%f').time()  for line in file if '2501' in line]
with open('/Users/zhenyu/Downloads/Loop2Info_2.txt') as file:
    end = [datetime.strptime(line.rstrip()[7:19], '%H:%M:%S.%f').time()  for line in file if '2215' in line]

In [74]:
time = 0
for i in range(500):
    time += (datetime.combine(date.today(), end_insert[i]) - datetime.combine(date.today(), start_insert[i])).total_seconds()

IndexError: list index out of range

In [70]:
time/1500

9.376666000000002

In [66]:
time = 0
for i in range(1500):
    time += (datetime.combine(date.today(), end[i]) - datetime.combine(date.today(), start[i])).total_seconds()

In [67]:
time/1500

104.73097466666677

In [59]:
path = '/Users/zhenyu/Downloads/ImageGeneration/Raw_Image/Template'
for file in os.listdir(path):
    if file.endswith('.jpg'):
        img = PIL.Image.open(os.path.join(path, file))
        resized_img = img.resize((189,252))
        resized_img.save(os.path.join(path,'T'+file))

In [83]:
ct = 0
for each in range(16, 25):
    df = pd.read_csv('/Users/zhenyu/Downloads/插线2/Insert2 V1.0_Summary_J327_E17_4_MP_2022_03_{}.csv'.format(each))
    df = df.loc[(df.CTOP < 20) & (df.CTOP > 6)]
    print('Cable inserting station 2 average on {} March is {}'.format(each, df.CTOP.mean()))
    ct += df.CTOP.mean()
print('Cable inserting station 2 average during last 9 days is {}'.format(ct/9))

Cable inserting station 2 average on 16 March is 6.999082644628104
Cable inserting station 2 average on 17 March is 7.4032
Cable inserting station 2 average on 18 March is 9.84142857142857
Cable inserting station 2 average on 19 March is 7.797587443946184
Cable inserting station 2 average on 20 March is 7.432223511214228
Cable inserting station 2 average on 21 March is 7.374715605250364
Cable inserting station 2 average on 22 March is 7.468218156227995
Cable inserting station 2 average on 23 March is 7.366854453627167
Cable inserting station 2 average on 24 March is 7.592160493827159
Cable inserting station 2 average during last 9 days is 7.697274542238865


In [75]:
df.CTOP.mean()

8.68808389585342

In [28]:
for root, dirs, files in os.walk('/Users/zhenyu/Desktop/AOI9/raw_filtered/ImageGeneration/Raw_Image'):
    for file in files:
        if file.endswith('.jpg') and len(file)>40:
            name_list.append(file)
    

In [32]:
df = pd.DataFrame(name_list)

In [27]:
len(file)

52

In [99]:
import coremltools
model_tf = coremltools.models.model.MLModel('/Users/zhenyu/Downloads/export.mlmodel')
model_torch = coremltools.models.model.MLModel('/Users/zhenyu/Desktop/results/bundle-415ce438b60241dd9a991d9e1bcb0196/weights/best.mlmodel')

In [100]:
desc_tf = model_tf.get_spec().description
desc_torch = model_torch.get_spec().description

In [101]:
model_tf.input_description

Features(input_1)

In [107]:
desc.input[0].type.multiArrayType.shape

[1, 1280, 1280, 3]

In [108]:
desc_torch.input[0].type.imageType

width: 960
height: 1280
colorSpace: RGB

In [127]:
import torch.nn as nn
from torchvision import transforms

In [131]:
image = PIL.Image.open('/Users/zhenyu/Documents/Scripts/IphoneAOI/yolov5/Template_Station2/FOV1/T01.jpg')
f = nn.AvgPool2d(2, stride=2)
trans = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                            transforms.ToTensor(), ])

img0 = trans(image)
img1 = img0[None]
for i in range(4):
    img1 = f(img1)

In [132]:
img0

tensor([[[0.60392, 0.55686, 0.69412,  ..., 0.15686, 0.14510, 0.18824],
         [0.61569, 0.52157, 0.36471,  ..., 0.15686, 0.14902, 0.18431],
         [0.58039, 0.47059, 0.56471,  ..., 0.15686, 0.15294, 0.18824],
         ...,
         [0.17647, 0.16863, 0.17255,  ..., 0.36863, 0.37647, 0.36471],
         [0.10196, 0.09804, 0.10196,  ..., 0.37647, 0.37255, 0.36471],
         [0.03529, 0.03529, 0.04314,  ..., 0.40000, 0.37255, 0.36863]]])

In [134]:
img0.shape

torch.Size([1, 90, 85])

In [136]:
img1

tensor([[[[0.68770, 0.74180, 0.72342, 0.69524, 0.50613],
          [0.71785, 0.72172, 0.72420, 0.70021, 0.50911],
          [0.72033, 0.70749, 0.74856, 0.74386, 0.52105],
          [0.68168, 0.74475, 0.78191, 0.75960, 0.48816],
          [0.66454, 0.70218, 0.71297, 0.64884, 0.28946]]]])

In [145]:
f1 = nn.AvgPool2d(16, stride=16)
img2 = img0[None]
img2 = f1(img2)

In [149]:
torch.squeeze(img2).numpy()

array([[     0.6877,     0.74181,     0.72342,     0.69524,     0.50613],
       [    0.71785,     0.72172,      0.7242,     0.70022,     0.50911],
       [    0.72033,     0.70749,     0.74856,     0.74386,     0.52105],
       [    0.68168,     0.74475,     0.78191,     0.75961,     0.48816],
       [    0.66454,     0.70218,     0.71298,     0.64884,     0.28946]], dtype=float32)

In [150]:
import cv2

In [212]:
img = PIL.Image.open('/Users/zhenyu/Documents/Scripts/IphoneAOI/yolov5/Template_Station2/FOV1.jpg')
temp = PIL.Image.open('/Users/zhenyu/Documents/Scripts/IphoneAOI/yolov5/Template_Station2/FOV1/T1.jpg')

In [190]:
res = cv2.matchTemplate(img, temp, cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

In [191]:
min_val, max_val, min_loc, max_loc

(-0.8389164209365845, 0.8417052030563354, (2246, 2877), (867, 323))

In [192]:
temp1 = cv2.imread('/Users/zhenyu/Documents/Scripts/IphoneAOI/yolov5/Template_Station2/FOV1/T1.jpg', cv2.IMREAD_GRAYSCALE)
f = nn.AvgPool2d(4, stride=4)
trans = transforms.Compose([transforms.ToTensor(),])

In [193]:
temp2 = f(trans(temp1))

In [206]:
temp3 = (torch.squeeze(temp2))*255

In [208]:
res = cv2.matchTemplate(img, np.array(temp3, dtype = np.uint8), cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

In [205]:
min_val, max_val, min_loc, max_loc

(-0.8388993144035339, 0.8416845798492432, (2246, 2877), (867, 323))

In [200]:
torch.squeeze(temp2).numpy().shape

(90, 85)

In [211]:
temp1.shape

(362, 341)

In [227]:
xy.append([img.width, img.height, temp.width, 0])

In [232]:
xyy = np.array(xy).transpose()

In [233]:
x1, y1, x2, y2 = xyy

In [234]:
x1

array([3024,    0, 3024, 3024, 3024])

In [235]:
xyy

array([[3024,    0, 3024, 3024, 3024],
       [4032, 4032,    0, 4032, 4032],
       [ 341,  341,  341,    0,  341],
       [ 362,  362,  362,  362,    0]])

In [243]:
xyyy = xyy[:,:4]

In [242]:
x01, y01, x02, y02, y

(4032, 4032, 0, 4032, 4032)

In [248]:
xyyy.reshape((8,2)).reshape((4,4))

array([[3024,    0, 3024, 3024],
       [4032, 4032,    0, 4032],
       [ 341,  341,  341,    0],
       [ 362,  362,  362,  362]])

In [247]:
xyyy

array([[3024,    0, 3024, 3024],
       [4032, 4032,    0, 4032],
       [ 341,  341,  341,    0],
       [ 362,  362,  362,  362]])

In [260]:
with open('template_idxs.yaml') as f:
    idxs, xy = yaml.load(f, Loader=Loader)

In [277]:
idxsn = np.array([idx1, idx2, idx3, idx4])

In [262]:
xy

array([[341, 362,   0, 434],
       [  0, 352,   0, 380],
       [  0, 411,   0,   0],
       [423, 447, 435,   0]])

In [278]:
idxsn

array([[[2788, 3521],
        [2760, 3448],
        [2756, 3487],
        [2684, 3476]],

       [[2145, 2777],
        [ 228,  836],
        [ 566, 1292],
        [1727, 2460]],

       [[2800, 3550],
        [2774, 3470],
        [ 977, 1534],
        [ 945, 1481]],

       [[ 445, 1044],
        [2011, 2563],
        [ 569, 1283],
        [1743, 2452]]])

In [272]:
idx=3
idxt1, idxt2, idxt3, idxt4 = idx1[idx], idx2[idx], idx3[idx], idx4[idx]

In [273]:
idxt1, idxt2, idxt3, idxt4

((2684, 3476), (1727, 2460), (945, 1481), (1743, 2452))

In [282]:
idxsn[:,3]

array([[2684, 3476],
       [1727, 2460],
       [ 945, 1481],
       [1743, 2452]])

In [ ]:
 P1 = (k1 + x01 - 1 + idxt2[0] - 1, h1 + y01 - 1 + idxt1[0] - 1)
    P2 = (k2 + x02 - 1 + idxt4[0] - 1, h2 + y02 - 1 + idxt3[0] - 1)